In [1]:
import os
train_address = "Train"

if not os.path.exists(train_address):
  os.makedirs(train_address)
  !mkdir images
  !pip install gdown
  !gdown https://drive.google.com/uc?id=1PYtBOmTYGVugPN1bMg51UCB1lf_cY_m-
  !unzip Center.zip
  !mv Center event
  !gdown https://drive.google.com/uc?id=1jEXejjSuRyPkGCobUgf3uHobXhhzzT5h
  !unzip Corner.zip
  !mv Corner event
  !gdown https://drive.google.com/uc?id=1b571zn4tTlPCFsTQlEp1MHxMEr-nVlQy
  !unzip Free-Kick.zip
  !mv Free-Kick event
  !gdown https://drive.google.com/uc?id=1FhnLqubUW_OS5V3KjEbIpDC2rjbkKiVn
  !unzip Left.zip
  !mv Left event
  !gdown https://drive.google.com/uc?id=1PBdkrqIwGorqRCTWIUoiWC8oKxCMsfnH
  !unzip Penalty.zip
  !mv Penalty event
  !gdown https://drive.google.com/uc?id=1ouS9-7c3eAFzMTkd1W9PLPBDkLjLIZGS
  !unzip Red-Cards.zip
  !mv Red-Cards event
  !gdown https://drive.google.com/uc?id=1pEjGFLmCAqK8blt2KSeUTtVDy6FQoZeR
  !unzip Right.zip
  !mv Right event
  !gdown https://drive.google.com/uc?id=17ZIFSj4gkfKjSz-GP9MF_vRU1nhpQLOd
  !unzip Tackle.zip
  !mv Tackle event
  !gdown https://drive.google.com/uc?id=1hpmmX2LwBM4OPrTazNTkEEOONeoGCpHb
  !unzip To-Subtitue.zip
  !mv "To Subtitue" event
  !gdown https://drive.google.com/uc?id=1HeI8akLuhJwJ9RWpYpAtHW5MKdgipFnm
  !unzip Yellow-Cards.zip
  !mv Yellow-Cards event
  !gdown https://drive.google.com/uc?id=14vP84hfX-dmuW07dnYSAFg_gZNFnIDuG
  !unzip Event.zip
  !mv Event SoccerEvent
  !gdown https://drive.google.com/uc?id=1XNwuiv1qGcNOwUoGZShZKbaJXn9N2_YV
  !unzip Others.zip
  !gdown https://drive.google.com/uc?id=1RiFOFQ9HSpzKgK3A_vbL05yIty5J6R-o
  !unzip Soccer.zip


In [2]:

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio
from PIL import Image
import pandas as pd
from IPython import display
import cv2
import pickle

In [ ]:
from imutils import paths
import random
from tensorflow.keras.preprocessing.image import img_to_array

filelist = "event/"
imagePaths = sorted(list(paths.list_images(filelist)))
random.seed(42)
random.shuffle(imagePaths)
data =[]
print(len(imagePaths))
cnt = 0
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
        if (cnt % 500) == 0 :
            print(cnt)
        if (cnt == 55000):
            break
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (60,80))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = img_to_array(image)
        data.append(image)
        cnt = cnt + 1



In [4]:
x=np.array(data)
x.shape

#randomly sample here!
np.random.shuffle(x)
train_images, test_images = np.split(x,[400])    #18000 is the train_size
#change the number of channels to 1.

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

train_images /= 255.
test_images /= 255.

batch_size = 64
train_size = len(train_images)
test_size = len(x) - train_size

#convert to tensorflow type tf.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(train_size).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices(test_images).shuffle(test_size).batch(batch_size)
train_dataset
#create a write
writer = tf.summary.create_file_writer('./new_loss')

In [5]:
#The Convolutional Variational Auto-Encoder Proper. 
#Define the network class

class CVAE(tf.keras.Model):
    def __init__(self, latent_dim):   #latent_dim is the dimensional of the latent space
        super(CVAE, self).__init__()
        self.latent_dim = latent_dim
        self.inference_net = tf.keras.Sequential(
          [
              tf.keras.layers.InputLayer(input_shape=(80,60, 3)),   #Note Input Image Size
              tf.keras.layers.Conv2D(
                  filters=8, kernel_size=3, strides=(2, 2), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=8, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=16, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=16, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=32, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=32, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=64, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=64, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Flatten(),
              # No activation
              tf.keras.layers.Dense(latent_dim + latent_dim),
          ]
        )

        self.generative_net = tf.keras.Sequential(
            [
              tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
              tf.keras.layers.Dense(units=40*30*32, activation=tf.nn.relu),
              tf.keras.layers.Reshape(target_shape=(40,30, 32)), #Note the strides formula!
              tf.keras.layers.Conv2DTranspose(
                  filters=8,
                  kernel_size=3,
                  strides=(2, 2),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=8,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=16,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=16,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=32,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=32,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=64,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=64,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              # No activation
              tf.keras.layers.Conv2DTranspose(
                  filters=3, kernel_size= 3, strides=(1, 1), padding="SAME", activation='sigmoid'),
            ]
        )
        
    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(64, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)

    def encode(self, x):   #encode the images into the latent space
        mean, logvar = tf.split(self.inference_net(x), num_or_size_splits=2, axis=1)
        return mean, logvar
    
    #the reparametrization trick suggests that we randomly sample 'e' from a unit Gaussian,
    #and then shift the randomly sampled 'e' by the latent distribution's mean 'mu' and scale it 
    #by the latent distribution's variance 'rho'.

    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    def decode(self, z, apply_sigmoid=False):  #decodes the latent space into the image
        logits = self.generative_net(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits

In [6]:
optimizer = tf.keras.optimizers.Adam(1e-4)

def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),axis=raxis)

@tf.function
def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = model.decode(z)

    kl_tolerance = 0.5
    #import pdb; pdb.set_trace()
    reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.square(x - x_logit), axis=[1, 2, 3]))
    kl_loss = - 0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar),axis=1)
    kl_loss = tf.reduce_mean(tf.maximum(kl_loss, kl_tolerance * latent_dim))
    loss_sum = reconstruction_loss + kl_loss
    
    with writer.as_default():
        tf.summary.scalar('Graph of' + '_reconstruction_loss', reconstruction_loss, step=epoch)
        tf.summary.scalar('Graph of' + '_kl_loss', kl_loss, step=epoch)
        tf.summary.scalar('Graph of' + '_loss_sum',loss_sum, step=epoch)
        writer.flush()
    return loss_sum, reconstruction_loss, kl_loss

@tf.function
def compute_apply_gradients(model, x, optimizer):
    with tf.GradientTape() as tape:
        loss, r_loss, kl_loss = compute_loss(model, x)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return r_loss, kl_loss

In [ ]:
epochs = 100
latent_dim = 128
num_examples_to_generate = 12

# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.

random_vector_for_generation = tf.random.normal(shape=[num_examples_to_generate, latent_dim])

model = CVAE(latent_dim)

def generate_and_save_images(model, epoch, test_input):
    predictions = model.sample(test_input)
    fig = plt.figure(figsize=(16,16))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0])
        plt.axis('off')

    # tight_layout minimizes the overlap between 2 sub-plots
    plt.savefig('./images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

def test_image(latent_dim,test_img):
    test_img = test_img.reshape(1,80,60,3)   #Note the Image size here
    model = CVAE(latent_dim)
    encod, logvar = model.encode(test_img)
    repar = model.reparameterize(encod, logvar)
    img = model.decode(repar)
    img *= 255.
    img = np.squeeze(img)
    return plt.imshow(img)

generate_and_save_images(model, 0, random_vector_for_generation)
df = pd.DataFrame()

losslist = []
for epoch in range(1, epochs + 1):
    start_time = time.time()
    for train_x in train_dataset:
        r_loss, kl_loss = compute_apply_gradients(model, train_x, optimizer)
        #print(r_loss, kl_loss)
    end_time = time.time()

    if epoch % 1 == 0:
        #loss = tf.keras.metrics.Mean()
        for test_x in test_dataset:
            loss_sum, rec_loss, kl_loss = compute_loss(model, test_x)
        losslist.append({'loss_sum':loss_sum.numpy(),'reconstruction_loss':rec_loss.numpy(),'kl_loss':kl_loss.numpy()})#append loss to list
        
        display.clear_output(wait=False)
        
        df = pd.DataFrame(losslist)  #append the dictionary to the dataframe
        df.to_csv('loss.csv')    #correct this

        print('Epoch: {}, Test set LOSS_sum: {}, reconstruction_loss: {}, kl_loss: {},'
              'time elapse for current epoch {}'.format(epoch,
                                                    loss_sum,rec_loss,kl_loss,
                                                    end_time - start_time))
    model.save_weights('my_model_weights.h5')
    generate_and_save_images(
        model, epoch, random_vector_for_generation)    


In [ ]:
def display_image(epoch_no):
    return PIL.Image.open('./images/image_at_epoch_{:04d}.png'.format(epoch_no))
plt.imshow(display_image(epochs))
plt.axis('off')# Display images    

In [ ]:
!cp -r event others
df = pd.read_csv('loss.csv',encoding = 'unicode_escape')
df = df.tail(300)

print(df)
df.plot(y='reconstruction_loss')
#import matplotlib.pyploy as plt

df.plot(y='kl_loss')
df.plot(y='loss_sum')


In [10]:
#The Convolutional Variational Auto-Encoder Proper. 
#Define the network class

class CVAE(tf.keras.Model):
    def __init__(self, latent_dim):   #latent_dim is the dimensional of the latent space
        super(CVAE, self).__init__()
        self.latent_dim = latent_dim
        self.inference_net = tf.keras.Sequential(
          [
              tf.keras.layers.InputLayer(input_shape=(80,60, 3)),   #Note Input Image Size
              tf.keras.layers.Conv2D(
                  filters=8, kernel_size=3, strides=(2, 2), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=8, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=16, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=16, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=32, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=32, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=64, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Conv2D(
                  filters=64, kernel_size=3, strides=(1, 1), activation='relu'),
              tf.keras.layers.Flatten(),
              # No activation
              tf.keras.layers.Dense(latent_dim + latent_dim),
          ]
        )

        self.generative_net = tf.keras.Sequential(
            [
              tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
              tf.keras.layers.Dense(units=40*30*32, activation=tf.nn.relu),
              tf.keras.layers.Reshape(target_shape=(40,30, 32)), #Note the strides formula!
              tf.keras.layers.Conv2DTranspose(
                  filters=8,
                  kernel_size=3,
                  strides=(2, 2),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=8,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=16,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=16,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=32,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=32,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=64,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              tf.keras.layers.Conv2DTranspose(
                  filters=64,
                  kernel_size=3,
                  strides=(1, 1),
                  padding="SAME",
                  activation='relu'),
              # No activation
              tf.keras.layers.Conv2DTranspose(
                  filters=3, kernel_size= 3, strides=(1, 1), padding="SAME", activation='sigmoid'),
            ]
        )
        
    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(64, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)

    def encode(self, x):   #encode the images into the latent space
        mean, logvar = tf.split(self.inference_net(x), num_or_size_splits=2, axis=1)
        return mean, logvar
    
    #the reparametrization trick suggests that we randomly sample 'e' from a unit Gaussian,
    #and then shift the randomly sampled 'e' by the latent distribution's mean 'mu' and scale it 
    #by the latent distribution's variance 'rho'.

    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    def decode(self, z, apply_sigmoid=False):  #decodes the latent space into the image
        logits = self.generative_net(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits

In [11]:
optimizer = tf.keras.optimizers.Adam(1e-4)

def log_normal_pdf(sample, mean, logvar, raxis=1):
    log2pi = tf.math.log(2. * np.pi)
    return tf.reduce_sum(-.5 * ((sample - mean) ** 2. * tf.exp(-logvar) + logvar + log2pi),axis=raxis)

@tf.function
def compute_loss(model, x):
    mean, logvar = model.encode(x)
    z = model.reparameterize(mean, logvar)
    x_logit = model.decode(z)

    kl_tolerance = 0.5
    #import pdb; pdb.set_trace()
    reconstruction_loss = tf.reduce_mean(tf.reduce_sum(tf.square(x - x_logit), axis=[1, 2, 3]))
    kl_loss = - 0.5 * tf.reduce_sum(1 + logvar - tf.square(mean) - tf.exp(logvar),axis=1)
    kl_loss = tf.reduce_mean(tf.maximum(kl_loss, kl_tolerance * latent_dim))
    loss_sum = reconstruction_loss + kl_loss
    
#     with writer.as_default():
#         tf.summary.scalar('Graph of' + '_reconstruction_loss', reconstruction_loss, step=epoch)
#         tf.summary.scalar('Graph of' + '_kl_loss', kl_loss, step=epoch)
#         tf.summary.scalar('Graph of' + '_loss_sum',loss_sum, step=epoch)
#         writer.flush()
    return loss_sum, reconstruction_loss, kl_loss

@tf.function
def compute_apply_gradients(model, x, optimizer):
    with tf.GradientTape() as tape:
        loss, r_loss, kl_loss = compute_loss(model, x)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return r_loss, kl_loss

In [12]:
epochs = 100
latent_dim = 256
num_examples_to_generate = 12

# keeping the random vector constant for generation (prediction) so
# it will be easier to see the improvement.

random_vector_for_generation = tf.random.normal(shape=[num_examples_to_generate, latent_dim])

def generate_and_save_images(model, epoch, test_input):
    predictions = model.sample(test_input)
    fig = plt.figure(figsize=(16,16))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0])
        plt.axis('off')

    # tight_layout minimizes the overlap between 2 sub-plots
    plt.savefig('./images/image_at_epoch_{:04d}.png'.format(epoch))
    plt.show()

In [ ]:
add = './Others/Others__1__0.jpg' 
test_inputs = Image.open(add)
newsize = (60, 80) 
test_inputs = test_inputs.resize(newsize) 
test_inputs = np.array(test_inputs)   
test_inputs = test_inputs.astype('float32')
test_inputs /= 255.


test_inputs = test_inputs.reshape(1,80,60,3)

#from tf.keras.models import load_model

def check_image(image,latent_dim):
    loaded_model = model
    #sample = model.sample()
    encod, logvar = loaded_model.encode(image)
    repar = loaded_model.reparameterize(encod, logvar)
    img = loaded_model.decode(repar)
    loss_sum, kl_loss, recon_loss = compute_loss(loaded_model, test_inputs)
    loss_sum, kl_loss, recon_loss = loss_sum.numpy(),recon_loss.numpy(),kl_loss.numpy()
    print(loss_sum, kl_loss, recon_loss)     #here for a group of pictures, we may want to 
    img = np.squeeze(img)
    return plt.imshow(img)

img = check_image(test_inputs,256)

img = check_image(test_inputs,256)
plt.axis('off')
plt.savefig('generative_example',quality = 95)



In [ ]:
folder = glob.glob('./Others/*')# test-image folder
loss_df = pd.DataFrame()

loss_list = []

def image_check_resize(folder,latent_dim):
    for file in folder:
        li = file.split('/')[2]
        name = li.split('.')[0]
        with open(file, 'r+b') as f:
            with Image.open(f) as image:
                cover = image.resize((60,80))
                test_inputs = np.array(cover)   
                test_inputs = test_inputs.astype('float32')
                test_inputs /= 255.
                test_inputs = test_inputs.reshape(1,80,60,3)
                loaded_model = model
                encod, logvar = loaded_model.encode(test_inputs)
                repar = loaded_model.reparameterize(encod, logvar)
                img = loaded_model.decode(repar)
                loss_sum, kl_loss, recon_loss = compute_loss(loaded_model, test_inputs)
                loss_list.append({'loss_sum':loss_sum.numpy(),'reconstruction_loss':recon_loss.numpy(),'kl_loss':kl_loss.numpy()}) #append loss to list   
                df = loss_df.append(loss_list, ignore_index=True)
                img = np.squeeze(img)
                im = Image.fromarray(np.uint8((img)*255))
                im = im.resize((640, 360))
                im.save('./others/{}.jpg'.format(name), image.format)
                df.to_csv('loss_recon_more_nft.csv')
    return plt.imshow(im)

show = image_check_resize(folder,latent_dim)

In [ ]:
folder = glob.glob('./Soccer/*')# test-image folder
loss_df = pd.DataFrame()

loss_list = []

def image_check_resize(folder,latent_dim):
    for file in folder:
        li = file.split('/')[2]
        name = li.split('.')[0]
        with open(file, 'r+b') as f:
            with Image.open(f) as image:
                cover = image.resize((60,80))
                test_inputs = np.array(cover)   
                test_inputs = test_inputs.astype('float32')
                test_inputs /= 255.
                test_inputs = test_inputs.reshape(1,80,60,3)
                loaded_model = model
                encod, logvar = loaded_model.encode(test_inputs)
                repar = loaded_model.reparameterize(encod, logvar)
                img = loaded_model.decode(repar)
                loss_sum, kl_loss, recon_loss = compute_loss(loaded_model, test_inputs)
                loss_list.append({'loss_sum':loss_sum.numpy(),'reconstruction_loss':recon_loss.numpy(),'kl_loss':kl_loss.numpy()}) #append loss to list   
                df = loss_df.append(loss_list, ignore_index=True)
                img = np.squeeze(img)
                im = Image.fromarray(np.uint8((img)*255))
                im = im.resize((640, 360))
                im.save('./others/{}.jpg'.format(name), image.format)
                df.to_csv('loss_recon_more_aft.csv')
    return plt.imshow(im)

show = image_check_resize(folder,latent_dim)

In [ ]:
folder = glob.glob('./SoccerEvent/*')# test-image folder
loss_df = pd.DataFrame()

loss_list = []

def image_check_resize(folder,latent_dim):
    for file in folder:
        li = file.split('/')[2]
        name = li.split('.')[0]
        with open(file, 'r+b') as f:
            with Image.open(f) as image:
                cover = image.resize((60,80))
                test_inputs = np.array(cover)   
                test_inputs = test_inputs.astype('float32')
                test_inputs /= 255.
                test_inputs = test_inputs.reshape(1,80,60,3)
                loaded_model = model
                encod, logvar = loaded_model.encode(test_inputs)
                repar = loaded_model.reparameterize(encod, logvar)
                img = loaded_model.decode(repar)
                loss_sum, kl_loss, recon_loss = compute_loss(loaded_model, test_inputs)
                loss_list.append({'loss_sum':loss_sum.numpy(),'reconstruction_loss':recon_loss.numpy(),'kl_loss':kl_loss.numpy()}) #append loss to list   
                df = loss_df.append(loss_list, ignore_index=True)
                img = np.squeeze(img)
                im = Image.fromarray(np.uint8((img)*255))
                im = im.resize((640, 360))
                im.save('./others/{}.jpg'.format(name), image.format)
                df.to_csv('loss_recon_more_ft.csv')
    return plt.imshow(im)

show = image_check_resize(folder,latent_dim)

In [26]:
df = pd.read_csv('loss_recon_more_ft.csv')
df1 = pd.read_csv('loss_recon_more_nft.csv')
df2 = pd.read_csv('loss_recon_more_aft.csv')

In [ ]:
ig1 = plt.figure(figsize=(8,8))
ax = plt.scatter(y = df['reconstruction_loss'], x = df['kl_loss'], color='b')  #football
ax1 = plt.scatter(y = df1['reconstruction_loss'], x = df1['kl_loss'], color='r')  #not_football
ax3 = plt.scatter(y = df2['reconstruction_loss'], x = df2['kl_loss'], color='g')  #american_football
plt.axhline(y=129, color='r', linestyle='--')
plt.legend((ax,ax1,ax3),('football','not_football','american_football'))
plt.ylabel('reconstruction_loss')
plt.xlabel('kl_loss')
plt.ylim((100,200))
#plt.savefig('various_rec_loss')

In [ ]:
cnt = 0
for i in range(0,1200):
  if (df['reconstruction_loss'][i] ) < 128 :
    cnt = cnt + 1
print(cnt)
cnt = 0
for i in range(0,1200):
  if (df1['reconstruction_loss'][i] ) < 128 :
    cnt = cnt + 1
print(cnt)
cnt = 0
for i in range(0,1200):
  if (df2['reconstruction_loss'][i] ) < 128 :
    cnt = cnt + 1
print(cnt)